### Lab: Create your collection of songs & audio features

To move forward witht the project, you need to create a collection of songs with their audio features - as large as possible! 

These are the songs that we will cluster. And, later, when the user inputs a song, we will find the cluster to which the song belongs and recommend a song from the same cluster.

The more songs you have, the more accurate and diverse recommendations you'll be able to give. Although... you might want to make sure the collected songs are "curated" in a certain way. Try to find playlists of songs that are diverse, but also that meet certain standards.

The process of sending hundreds or thousands of requests can take some time - it's normal if you have to wait a few minutes (or, if you're ambitious, even hours) to get all the data you need.

An idea for collecting as many songs as possible is to start with all the songs of a big, diverse playlist and then go to every artist present in the playlist and grab every song of every album of that artist. The amount of songs you'll be collecting per playlist will grow exponentially!

## Spotipy API

Create an Spotify account and follow these steps to register an app: https://developer.spotify.com/documentation/general/guides/app-settings/

After the app is created, you can see it on your dashboard
https://developer.spotify.com/dashboard/applications

Click on it and you'll find the client id and client secret.

#### Authentification

In [1]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import json
import pprint

In [2]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="f26a41dabb5a42a69f618e86fcf8f64b",
    client_secret="068ab1c712354fef9081383b94a7f95a"))

#### Pagination using "next"

When you collect songs from a playlist using `sp.playlist_tracks`, you're limited by the `limit` parameter, which has a maximum (and default) value of 100. When the playlist has more than 100 songs, you have to collect them by navigating through the "pages" of the results.

The parameter `offset` allows you to retrieve resuls starting at a certain position: if you start at position 101, you'd get the next "page" of results. An offset of 201 would give you the third page, and so on.

The function `sp.next()` does the same, but in a simpler way: it can be used on the results from any request to directly retrieve the results for the next page.

We can check whether there's a next page or not by accessing the key `next` on the results from any request.

In [3]:
tracks_from_playlist = sp.playlist_tracks("https://open.spotify.com/playlist/6YvswZI0jzYxLnaYN42Ptg")

In [4]:
# this is the link to the "next" page
next_page_link = tracks_from_playlist["next"]
next_page_link

'https://api.spotify.com/v1/playlists/6YvswZI0jzYxLnaYN42Ptg/tracks?offset=100&limit=100&additional_types=track'

In [5]:
# this gives the full results directly
next_results = sp.next(tracks_from_playlist)
next_results

{'href': 'https://api.spotify.com/v1/playlists/6YvswZI0jzYxLnaYN42Ptg/tracks?offset=100&limit=100&additional_types=track',
 'items': [{'added_at': '2016-01-01T09:12:39Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/jakesn55'},
    'href': 'https://api.spotify.com/v1/users/jakesn55',
    'id': 'jakesn55',
    'type': 'user',
    'uri': 'spotify:user:jakesn55'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/776Uo845nYHJpNaStv1Ds4'},
       'href': 'https://api.spotify.com/v1/artists/776Uo845nYHJpNaStv1Ds4',
       'id': '776Uo845nYHJpNaStv1Ds4',
       'name': 'Jimi Hendrix',
       'type': 'artist',
       'uri': 'spotify:artist:776Uo845nYHJpNaStv1Ds4'}],
     'available_markets': ['AD',
      'AE',
      'AL',
      'AR',
      'AT',
      'AU',
      'BA',
      'BE',
      'BG',
      'BH',
      'BO',
      'BR',
      'BY',


In [6]:
# this playlist has 111 songs, so there's no 3rt page:
print(sp.next(next_results))
print(next_results["next"])

{'href': 'https://api.spotify.com/v1/playlists/6YvswZI0jzYxLnaYN42Ptg/tracks?offset=200&limit=100&additional_types=track', 'items': [{'added_at': '2016-01-01T12:45:06Z', 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/jakesn55'}, 'href': 'https://api.spotify.com/v1/users/jakesn55', 'id': 'jakesn55', 'type': 'user', 'uri': 'spotify:user:jakesn55'}, 'is_local': False, 'primary_color': None, 'track': {'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5KnLt0VENhwsfmQL3y2nsY'}, 'href': 'https://api.spotify.com/v1/artists/5KnLt0VENhwsfmQL3y2nsY', 'id': '5KnLt0VENhwsfmQL3y2nsY', 'name': 'Michel Fugain & Le Big Bazar', 'type': 'artist', 'uri': 'spotify:artist:5KnLt0VENhwsfmQL3y2nsY'}], 'available_markets': ['AD', 'AE', 'AL', 'AR', 'AT', 'BA', 'BE', 'BG', 'BH', 'BO', 'BR', 'BY', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HN', 'HR', 'HU'

 ## Sample lab solution

#### Step 1: get uris of all the songs in a playlist (no matter how many songs the playlist has)

In [7]:
def get_tracks_ids_from_playlist(playlist_id):
    
    final_results = []
    result = sp.playlist_tracks(playlist_id)
    final_results.extend(result["items"])
    
    while result["next"]:
        
        result = sp.next(result)
        final_results.extend(result["items"])
        
    ids = [track["track"]["id"] for track in final_results]
    
    # we return only values that are not None:
    return [i for i in ids if i]

In [8]:
get_tracks_ids_from_playlist("https://open.spotify.com/playlist/6YvswZI0jzYxLnaYN42Ptg")

['7pKfPomDEeI4TPT6EOYjn9',
 '1AhDOtG9vPSOmsWgNW0BEY',
 '40riOy7x9W7GXjyGp4pjAv',
 '51pQ7vY7WXzxskwloaeqyj',
 '70C4NyhjD5OZUMzvWZ3njJ',
 '3n3F07lHLyRKwqg4q64eYA',
 '1wHSxWBfPr3GO31y8KGrWe',
 '53968oKecrFxkErocab2Al',
 '2dWCp5vcNwSy25zRVFGC5t',
 '7EmSPZ9f1AiLR9eVHNSI62',
 '7HrzErXq3TsKOY1gmdIShB',
 '6QPKYGnAW9QozVz2dSWqRg',
 '6iGU74CwXuT4XVepjc9Emf',
 '3PKtemUKxiDBvBo7tpQ8bG',
 '3LTMnFa0hhwisyq6ILahyj',
 '5S6pYZD8WH3OQapDHTDM9M',
 '0BCPKOYdS2jbQ8iyB56Zns',
 '6lveWvGIUSw4ZRqBzTU9ia',
 '1f3yAtsJtY87CTmM8RLnxf',
 '75JFxkI2RXiU7L9VXzMkle',
 '3mr584ZFSkZrWja9kJAzlb',
 '0aKZS2sM8zI9I6Esp420V3',
 '0FeCO85RKW8fDRytwXof2x',
 '0T6kwiueP62ten2KLLmQS4',
 '6TSP9oSbXONHk39GHvOSs3',
 '2LkaNhCrNVmcYgXJeLVmsw',
 '29r02BhNetddIKu8xdRigJ',
 '14XWXWv5FoCbFzLksawpEe',
 '5V1AHQugSTASVez5ffJtFo',
 '4j6rSruR2yQO6dOkf0OXbU',
 '4wYq5wugZDzQiMZQYG4wVB',
 '4kAJawkLZj4Q7N5521ofP6',
 '5gFkAC68P66ZNVKXKjYtDm',
 '2RlgNHKcydI9sayD2Df2xp',
 '6l8EbYRtQMgKOyc1gcDHF9',
 '5j6ZZwA9BnxZi5Bk0Ng4jB',
 '1L94M3KIu7QluZe63g64rv',
 

#### Step 2: build a df with all the audio features

In [12]:
# we're gonna iterate through the output of the function above
#sp.audio_features(track_ids[0])[0]

In [10]:
def get_audio_features_df(playlist_id):
    track_ids = get_tracks_ids_from_playlist(playlist_id)
    aud_feat = [sp.audio_features(track)[0] for track in track_ids]
    return pd.DataFrame(aud_feat)

In [14]:
audio_f = get_audio_features_df("https://open.spotify.com/playlist/6uYt5DwcyaOxxPUundboC2")
audio_f.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.339,0.143,0,-10.780,1,0.0317,0.921000,0.000636,0.3720,0.0948,101.213,audio_features,74X1epeRufHckhuX1KFD04,spotify:track:74X1epeRufHckhuX1KFD04,https://api.spotify.com/v1/tracks/74X1epeRufHc...,https://api.spotify.com/v1/audio-analysis/74X1...,413320,3
1,0.377,0.682,7,-8.039,1,0.0299,0.000894,0.002170,0.0504,0.2880,108.736,audio_features,1QEEqeFIZktqIpPI4jSVSF,spotify:track:1QEEqeFIZktqIpPI4jSVSF,https://api.spotify.com/v1/tracks/1QEEqeFIZktq...,https://api.spotify.com/v1/audio-analysis/1QEE...,285133,4
2,0.138,0.712,0,-5.271,1,0.0690,0.746000,0.773000,0.0714,0.1330,99.877,audio_features,6m4HWTYMRSJkaUuvXTaNmE,spotify:track:6m4HWTYMRSJkaUuvXTaNmE,https://api.spotify.com/v1/tracks/6m4HWTYMRSJk...,https://api.spotify.com/v1/audio-analysis/6m4H...,203067,4
3,0.573,0.902,6,-7.638,0,0.0540,0.018000,0.001250,0.0928,0.8760,84.412,audio_features,2WfaOiMkCvy7F5fcp2zZ8L,spotify:track:2WfaOiMkCvy7F5fcp2zZ8L,https://api.spotify.com/v1/tracks/2WfaOiMkCvy7...,https://api.spotify.com/v1/audio-analysis/2Wfa...,225280,4
4,0.630,0.519,9,-10.997,1,0.0528,0.225000,0.000004,0.0974,0.5020,143.942,audio_features,0sDqo9UPzPUtu9wEkI3zRB,spotify:track:0sDqo9UPzPUtu9wEkI3zRB,https://api.spotify.com/v1/tracks/0sDqo9UPzPUt...,https://api.spotify.com/v1/audio-analysis/0sDq...,278627,4


#### Step 3: grow the df with multiple playlists

In [17]:
"""playlists = ["https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF",#global top 50
             "https://open.spotify.com/playlist/37i9dQZF1DX2apWzyECwyZ",#bad bunny
             "https://open.spotify.com/playlist/2GjnnR5TSgsNcSWR3rSLtf",#the weekend
             "https://open.spotify.com/playlist/37i9dQZF1DX1PfYnYcpw8w",#ariana grande
             "https://open.spotify.com/playlist/37i9dQZF1DX3lQi4RhMuwd",#maria carey
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO3GXIhX",#24kgolden
             "https://open.spotify.com/playlist/37i9dQZF1DX08mhnhv6g9b",#bts
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO38Wlm8",#wham!
             "https://open.spotify.com/playlist/37i9dQZF1DX6cg4h2PoN9y",#billie eilish
             "https://open.spotify.com/playlist/37i9dQZF1DX7JVsTTEkyUT",#karol g
             "https://open.spotify.com/playlist/37i9dQZF1DWX37VefS7A1r",#shawn mendes
             "https://open.spotify.com/playlist/37i9dQZF1DX5kVmFQxhnLW",#michael buble
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO3ZsXzq",#harry styles
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO04kFZs",#pop smoke
             "https://open.spotify.com/playlist/37i9dQZF1DXa0PTjSQ7AeJ",#miley cyrus
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2st71e",#brenda lee
             "https://open.spotify.com/playlist/37i9dQZF1DXc2aPBXGmXrt",#justin bieber
             "https://open.spotify.com/playlist/37i9dQZF1DWXXQuxfjYVxb",#cardi b
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO1OE2SA",#bobby helms
             "https://open.spotify.com/playlist/37i9dQZF1DX66AtErk5xTK",#maluma
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2nSXaa",#tate mcrae
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2BMiFW",#andy williams
             "https://open.spotify.com/playlist/37i9dQZF1DXbdhJjehriAj",#dababy
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO3UZtDi",#joel corry
             "https://open.spotify.com/playlist/37i9dQZF1DWZbUTR2krEvE",#kelly clarkson
             "https://open.spotify.com/playlist/37i9dQZF1DWX2qUCdViQ0N",#lewis capaldi
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2SyYJ7",#connan grey
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO0lqudC",#saint jhn
             "https://open.spotify.com/playlist/37i9dQZF1DX3fRquEp6m8D",#dua lipa
             "https://open.spotify.com/playlist/37i9dQZF1DWVx0p3emp3c6",#sech
             "https://open.spotify.com/playlist/37i9dQZF1DWVxabNKXcEZh",#manuel turiza
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2qiE6s",#dean martin
             "https://open.spotify.com/playlist/37i9dQZF1DX7QOv5kjbU68"]#drake

In [28]:
playlists = ["https://open.spotify.com/playlist/37i9dQZEVXbMDoHDwVN2tF",#global top 50
             "https://open.spotify.com/playlist/37i9dQZF1DX2apWzyECwyZ",#bad bunny
             "https://open.spotify.com/playlist/2GjnnR5TSgsNcSWR3rSLtf",#the weekend
             "https://open.spotify.com/playlist/37i9dQZF1DX1PfYnYcpw8w",#ariana grande
             "https://open.spotify.com/playlist/37i9dQZF1DX3lQi4RhMuwd",#maria carey
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO3GXIhX",#24kgolden
             "https://open.spotify.com/playlist/37i9dQZF1DX08mhnhv6g9b",#bts
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO38Wlm8",#wham!
             "https://open.spotify.com/playlist/37i9dQZF1DX6cg4h2PoN9y",#billie eilish
             "https://open.spotify.com/playlist/37i9dQZF1DX7JVsTTEkyUT",#karol g
             "https://open.spotify.com/playlist/37i9dQZF1DWX37VefS7A1r",#shawn mendes
             "https://open.spotify.com/playlist/37i9dQZF1DX5kVmFQxhnLW",#michael buble
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO3ZsXzq",#harry styles
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO04kFZs",#pop smoke
             "https://open.spotify.com/playlist/37i9dQZF1DXa0PTjSQ7AeJ",#miley cyrus
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2st71e"]#brenda lee

In [ ]:
playlists2 = ["https://open.spotify.com/playlist/37i9dQZF1DXc2aPBXGmXrt",#justin bieber
             "https://open.spotify.com/playlist/37i9dQZF1DWXXQuxfjYVxb",#cardi b
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO1OE2SA",#bobby helms
             "https://open.spotify.com/playlist/37i9dQZF1DX66AtErk5xTK",#maluma
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2nSXaa",#tate mcrae
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2BMiFW",#andy williams
             "https://open.spotify.com/playlist/37i9dQZF1DXbdhJjehriAj",#dababy
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO3UZtDi",#joel corry
             "https://open.spotify.com/playlist/37i9dQZF1DWZbUTR2krEvE",#kelly clarkson
             "https://open.spotify.com/playlist/37i9dQZF1DWX2qUCdViQ0N",#lewis capaldi
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2SyYJ7",#connan grey
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO0lqudC",#saint jhn
             "https://open.spotify.com/playlist/37i9dQZF1DX3fRquEp6m8D",#dua lipa
             "https://open.spotify.com/playlist/37i9dQZF1DWVx0p3emp3c6",#sech
             "https://open.spotify.com/playlist/37i9dQZF1DWVxabNKXcEZh",#manuel turiza
             "https://open.spotify.com/playlist/37i9dQZF1DZ06evO2qiE6s",#dean martin
             "https://open.spotify.com/playlist/37i9dQZF1DX7QOv5kjbU68"]#drake

In [31]:
def get_big_audio_df(playlists_list):
    audio_f_df = pd.DataFrame()
    
    for p in playlists_list:
        audio_f_df = audio_f_df.append(get_audio_features_df(p))

    return audio_f_df

In [32]:
big_df_songs_audiofeat = get_big_audio_df(playlists)

In [33]:
big_df_songs_audiofeat

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.731,0.5730,4,-10.059,0,0.0544,0.40100,0.000052,0.1130,0.145,109.928,audio_features,4MzXwWMhyBbmu6hOcLVD49,spotify:track:4MzXwWMhyBbmu6hOcLVD49,https://api.spotify.com/v1/tracks/4MzXwWMhyBbm...,https://api.spotify.com/v1/audio-analysis/4MzX...,205090,4
1,0.514,0.7300,1,-5.934,1,0.0598,0.00146,0.000095,0.0897,0.334,171.005,audio_features,0VjIjW4GlUZAMYd2vXMi3b,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,https://api.spotify.com/v1/tracks/0VjIjW4GlUZA...,https://api.spotify.com/v1/audio-analysis/0VjI...,200040,4
2,0.737,0.8020,0,-4.771,1,0.0878,0.46800,0.000000,0.0931,0.682,144.015,audio_features,35mvY5S1H3J2QZyna3TFe0,spotify:track:35mvY5S1H3J2QZyna3TFe0,https://api.spotify.com/v1/tracks/35mvY5S1H3J2...,https://api.spotify.com/v1/audio-analysis/35mv...,172325,4
3,0.336,0.6270,7,-7.463,1,0.0384,0.16400,0.000000,0.0708,0.350,150.273,audio_features,0bYg9bo50gSsH3LtXe2SQn,spotify:track:0bYg9bo50gSsH3LtXe2SQn,https://api.spotify.com/v1/tracks/0bYg9bo50gSs...,https://api.spotify.com/v1/audio-analysis/0bYg...,241107,4
4,0.700,0.7220,7,-3.558,0,0.0369,0.22100,0.000000,0.2720,0.756,90.989,audio_features,3tjFYV6RSFtuktYl3ZtYcq,spotify:track:3tjFYV6RSFtuktYl3ZtYcq,https://api.spotify.com/v1/tracks/3tjFYV6RSFtu...,https://api.spotify.com/v1/audio-analysis/3tjF...,140526,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,0.405,0.0651,8,-10.664,1,0.0335,0.80700,0.000000,0.1040,0.380,88.945,audio_features,5yAoVFuUY0uLcbiUyhM6D5,spotify:track:5yAoVFuUY0uLcbiUyhM6D5,https://api.spotify.com/v1/tracks/5yAoVFuUY0uL...,https://api.spotify.com/v1/audio-analysis/5yAo...,160973,3
46,0.767,0.5250,2,-5.643,0,0.0302,0.30800,0.000000,0.3940,0.578,98.311,audio_features,0Lal493pXMojcVT1WQeiQo,spotify:track:0Lal493pXMojcVT1WQeiQo,https://api.spotify.com/v1/tracks/0Lal493pXMoj...,https://api.spotify.com/v1/audio-analysis/0Lal...,140954,4
47,0.664,0.5240,0,-7.258,1,0.0457,0.29900,0.000174,0.1370,0.734,82.412,audio_features,2lAA9mRv18zcZFNblZVAVw,spotify:track:2lAA9mRv18zcZFNblZVAVw,https://api.spotify.com/v1/tracks/2lAA9mRv18zc...,https://api.spotify.com/v1/audio-analysis/2lAA...,125893,4
48,0.325,0.3260,4,-11.734,1,0.0261,0.38400,0.000082,0.0742,0.185,97.622,audio_features,7aIm08TUlgs3g6b9nP1aTy,spotify:track:7aIm08TUlgs3g6b9nP1aTy,https://api.spotify.com/v1/tracks/7aIm08TUlgs3...,https://api.spotify.com/v1/audio-analysis/7aIm...,248733,3


Disclaimer: when Spotify's API takes more than 5 seconds to respond, it gives a Time readout error - ideally we'd use try - except to circumvent that. It can be in our next iteration!